In [1]:
# Imports
import sys
from pathlib import Path
import tiktoken
import os

sys.path.append(str(Path(os.getcwd()).parent))
'''
import openai

openai.api_key = ""
openai.api_base = "https://neuroapi.host"
'''
import g4f
g4f.logging = False

In [2]:
# Count tokens
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [3]:
# New session
system_message = {"role": "system", "content": "Ты очень полезный ИИ помочник."}
max_response_tokens = 1024
token_limit = 16000
conversation = []
conversation.append(system_message)

In [4]:
# User input
stream = True
while True:
    user_input = input("")
    if user_input == "exit":
        break
    elif user_input == "clear":
        conversation = []
        conversation.append(system_message)
        continue
    elif user_input == "regenerate":
        del conversation[-1]
    else:
        conversation.append({"role": "user", "content": user_input})
    conv_history_tokens = num_tokens_from_messages(conversation)

    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)

    while True:
        try:
            chat_completion = g4f.ChatCompletion.create(
                model="gpt-3.5-turbo-16k-0613",
                messages=conversation,
                stream=stream,
            )

            if not stream:
                # not stream
                if chat_completion == "":
                    continue
                print(chat_completion)
                conversation.append({"role": "system", "content": chat_completion})
            else:
                # stream
                collected_message = ""
                for message in chat_completion:
                    if message == "Using NeuroGPT provider" or "Invalid" in message:
                        continue
                    print(message, flush=True, end="")
                    collected_message += message
                print()
                if collected_message == "":
                    continue
                conversation.append({"role": "assistant", "content": collected_message})
            break
        except Exception as e:
            print(e)


Здравствуйте! Чем я могу Вам помочь?


In [5]:
# Print history
for message in conversation:
    print(message)

{'role': 'system', 'content': 'Ты очень полезный ИИ помочник.'}
{'role': 'user', 'content': 'Привет'}
{'role': 'assistant', 'content': 'Здравствуйте! Чем я могу Вам помочь?'}
